### Data Ingestion

In [57]:
from langchain_core.documents import Document

In [58]:
doc=Document(
    page_content="this is the main text content I am using to create RAG",
    metadata={
        "source":"exmaple.txt",
        "pages":1,
        "author":"Pruthviraj Rathod",
        "date_created":"2025-01-01"
    }
)
doc

Document(metadata={'source': 'exmaple.txt', 'pages': 1, 'author': 'Pruthviraj Rathod', 'date_created': '2025-01-01'}, page_content='this is the main text content I am using to create RAG')

In [59]:
import os
os.makedirs("../data/text_files",exist_ok=True)

In [60]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [61]:
from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader=TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [62]:
from langchain_community.document_loaders import (
    DirectoryLoader,
    TextLoader,
    PyPDFLoader,
    CSVLoader,
    UnstructuredWordDocumentLoader,
    JSONLoader,
    SQLDatabaseLoader
)
from langchain_community.utilities import SQLDatabase


# ---------------- FILE LOADERS ---------------- #
LOADER_MAPPING = {
    ".txt": (TextLoader, {"encoding": "utf-8"}),
    ".pdf": (PyPDFLoader, {}),
    ".csv": (CSVLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".json": (JSONLoader, {"jq_schema": "."}),
}


def load_file_documents(data_dir: str = "../data"):
    """Load all supported files from a directory"""
    all_documents = []
    for ext, (loader_cls, loader_kwargs) in LOADER_MAPPING.items():
        loader = DirectoryLoader(
            data_dir,
            glob=f"**/*{ext}",
            loader_cls=loader_cls,
            loader_kwargs=loader_kwargs,
            show_progress=True,
        )
        docs = loader.load()
        all_documents.extend(docs)
        print(f"Loaded {len(docs)} {ext} documents")
    return all_documents


# ---------------- DATABASE LOADER ---------------- #
def load_db_documents(db_uri: str, query: str = "SELECT * FROM documents"):
    """Load documents from a SQL database"""
    db = SQLDatabase.from_uri(db_uri)
    loader = SQLDatabaseLoader(database=db, query=query)
    docs = loader.load()
    print(f"Loaded {len(docs)} documents from database")
    return docs


# ---------------- COMBINED LOADER ---------------- #
def load_all_sources(
    data_dir: str = "../data",
    db_uri: str = None,
    query: str = "SELECT * FROM documents"
):
    documents = []

    # Load files
    file_docs = load_file_documents(data_dir)
    documents.extend(file_docs)

    # Load from database (if provided)
    if db_uri:
        db_docs = load_db_documents(db_uri, query=query)
        documents.extend(db_docs)

    print(f"\n✅ Total documents loaded: {len(documents)}")
    return documents



In [63]:
documents

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '),
 Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popu

In [64]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [65]:
from pathlib import Path
from typing import List
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    CSVLoader,
    UnstructuredWordDocumentLoader,
    JSONLoader,
    SQLDatabaseLoader
)
from sqlalchemy import create_engine
from langchain.schema import Document

# Map file extensions to loaders
LOADER_MAPPING = {
    ".pdf": PyPDFLoader,
    ".txt": TextLoader,
    ".csv": CSVLoader,
    ".docx": UnstructuredWordDocumentLoader,
    ".json": JSONLoader,
}

def load_documents(data_dir: str = "../data") -> List[Document]:
    """Load all supported files from a directory into Documents"""
    all_documents = []
    data_path = Path(data_dir)

    for file in data_path.rglob("*"):
        if not file.is_file():
            continue

        ext = file.suffix.lower()
        if ext in LOADER_MAPPING:
            loader_cls = LOADER_MAPPING[ext]
            try:
                loader = loader_cls(str(file))
                docs = loader.load()
                for d in docs:
                    d.metadata["source_file"] = file.name
                    d.metadata["file_type"] = ext
                all_documents.extend(docs)
                print(f"✅ Loaded {len(docs)} docs from {file.name}")
            except Exception as e:
                print(f"❌ Error loading {file.name}: {e}")
        else:
            print(f"⚠️ Skipping unsupported file: {file.name}")

    print(f"\n📦 Total documents loaded: {len(all_documents)}")
    return all_documents


# Example: Database ingestion
def load_from_database(conn_str: str, query: str) -> List[Document]:
    """Load documents from a database query"""
    try:
        loader = SQLDatabaseLoader(
            create_engine(conn_str),
            query=query
        )
        docs = loader.load()
        for d in docs:
            d.metadata["source_file"] = "database"
            d.metadata["file_type"] = "db"
        print(f"✅ Loaded {len(docs)} docs from database")
        return docs
    except Exception as e:
        print(f"❌ DB error: {e}")
        return []


# ---------------- Run ----------------
all_documents = load_documents("../data")
# db_documents = load_from_database("sqlite:///mydb.sqlite", "SELECT * FROM my_table")


⚠️ Skipping unsupported file: my_database.db


✅ Loaded 60 docs from Ebook-Agentic-AI.pdf
✅ Loaded 1 docs from machine_learning.txt
✅ Loaded 1 docs from python_intro.txt
⚠️ Skipping unsupported file: chroma.sqlite3
⚠️ Skipping unsupported file: data_level0.bin
⚠️ Skipping unsupported file: header.bin
⚠️ Skipping unsupported file: length.bin
⚠️ Skipping unsupported file: link_lists.bin

📦 Total documents loaded: 62


In [66]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [67]:
chunks=split_documents(all_pdf_documents)
chunks

Split 60 documents into 119 chunks

Example chunk:
Content: Konverge AI is a decision science firm empowering 
businesses with the transformative power of AI. 
Operating at the intersection of data, machine 
learning (ML) models, and business insights, we 
hel...
Metadata: {'source': '..\\data\\pdf\\Ebook-Agentic-AI.pdf', 'page': 1, 'source_file': 'Ebook-Agentic-AI.pdf', 'file_type': 'pdf'}


[Document(metadata={'source': '..\\data\\pdf\\Ebook-Agentic-AI.pdf', 'page': 1, 'source_file': 'Ebook-Agentic-AI.pdf', 'file_type': 'pdf'}, page_content='Konverge AI is a decision science firm empowering \nbusinesses with the transformative power of AI. \nOperating at the intersection of data, machine \nlearning (ML) models, and business insights, we \nhelp organizations develop cutting-edge AI \nproducts and solutions. \nThis book provides actionable insights into Agentic \nAI, combining Konverge AI’s expertise with \ncontributions from Emergence AI. Emergence AI \nshared its deep knowledge in autonomous \nmulti-agent orchestration, addressing challenges \nlike outdated systems, complex processes, and \nregulatory compliance. Their solutions enhance \nadaptability and optimize operations.\nBy merging these perspectives, this book is a \npractical guide to leveraging Agentic AI, helping \nbusinesses navigate and excel in a dynamic world.'),
 Document(metadata={'source': '..\\data\\pdf\

In [68]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


Model loaded successfully. Embedding dimension: 384


In [70]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 238


In [71]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings 

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 119 texts...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches: 100%|██████████| 4/4 [00:06<00:00,  1.58s/it]


Generated embeddings with shape: (119, 384)
Adding 119 documents to vector store...
Successfully added 119 documents to vector store
Total documents in collection: 357


Retriever Pipeline From VectorStore


In [72]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [73]:
rag_retriever

In [79]:
rag_retriever.retrieve("What is agentic  ai ")

Retrieving documents for query: 'What is agentic  ai '
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.84it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_3314b696_1',
  'content': "Agentic AI\nAn Executive's Guide to In-depth\nUnderstanding of Agentic AI",
  'metadata': {'source_file': 'Ebook-Agentic-AI.pdf',
   'doc_index': 1,
   'content_length': 71,
   'source': '..\\data\\pdf\\Ebook-Agentic-AI.pdf',
   'page': 2,
   'file_type': 'pdf'},
  'similarity_score': 0.5667293965816498,
  'distance': 0.4332706034183502,
  'rank': 1},
 {'id': 'doc_30a51656_1',
  'content': "Agentic AI\nAn Executive's Guide to In-depth\nUnderstanding of Agentic AI",
  'metadata': {'page': 2,
   'doc_index': 1,
   'file_type': 'pdf',
   'source': '..\\data\\pdf\\Ebook-Agentic-AI.pdf',
   'content_length': 71,
   'source_file': 'Ebook-Agentic-AI.pdf'},
  'similarity_score': 0.5667293965816498,
  'distance': 0.4332706034183502,
  'rank': 2},
 {'id': 'doc_6fd5599d_1',
  'content': "Agentic AI\nAn Executive's Guide to In-depth\nUnderstanding of Agentic AI",
  'metadata': {'file_type': 'pdf',
   'doc_index': 1,
   'source': '..\\data\\pdf\\Ebook-Agentic